In [1]:
import cv2
import glob,os
import numpy as np
from tqdm import tqdm
import json
from myutil import *

from mydataloader import FrameProcessor, YoloLoader, PoseModelLoader

class myVideoLoader(object):
    def __init__(self, videoPath):
        self.videoPath = videoPath
        self.dirPath , self.videoname = os.path.split(videoPath)
        self.cap = cv2.VideoCapture(videoPath)
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS)) 
        self.fcount = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
    def get_audio(self):
        cap = cv2.VideoCapture(self.videoPath)
        ret, frame = cap.read()
        self.audioPath = self.videoPath[:-4]+'.wav'
        command = "ffmpeg -i {} -vn -ar 16000 -f wav {}".format(self.videoPath, self.audioPath)
        try:
            subprocess.check_call(command, shell=True)
            is_success = True
        except subprocess.CalledProcessError as e:
            print("error code: {}! shell command: {}".format(e.returncode, e.cmd))
            is_success = False
        assert is_success==True, print('convert error!')
        cap.release()
        return self.audioPath
     
    
    def extract_frame(self, savDir, by_sec=1, useFilter = True, min_det_num = 40, filter_jump_sec=5):
        #by_sec=5 : save frame every 5 second 
        #useFilter, min_det_num : use alphapose to detect person number and filter the frame (detect_num<min_det_num)
        #filter_jump_sec : if detect frame, jump x sec.
        if not os.path.exists(savDir):
            os.makedirs(savDir)
            
        if useFilter:
            # Load model
            pose_model_loader = PoseModelLoader()
            det_model_loader = YoloLoader()
            
        cap = cv2.VideoCapture(self.videoPath)
        fI_jump=0
        for fI in tqdm(range(0,self.fcount,self.fps*by_sec)):
            if fI<fI_jump:
                continue
            cap.set(cv2.CAP_PROP_POS_FRAMES,fI)
            ret, frame = cap.read()
            if useFilter:
                framePose = FrameProcessor(frame, det_model_loader, pose_model_loader).start()
                result = framePose.resultNew
                person_num = framePose.det_human_num
                if person_num<min_det_num:
                    continue
            imgSavname=self.videoname[:-4]+'_'+str(fI)+'_'+str(person_num)+'.jpg'
            jsonSavname=self.videoname[:-4]+'_'+str(fI)+'_'+str(person_num)+'.json'
            sav_img(os.path.join(savDir,self.videoname[:-4],imgSavname), frame)
            with open(os.path.join(savDir,self.videoname[:-4],jsonSavname),'w',encoding='utf-8') as jsonfile:
                json.dump(result, jsonfile)
            fI_jump = fI + self.fps*filter_jump_sec
            #print(person_num, savname)
        cap.release() 
    
    def auto_extract_frame(self, savDir, by_sec=1, filter_jump_sec=5):
        imgSavDir = os.path.join(savDir,self.videoname[:-4],'images')
        jsonSavDir = os.path.join(savDir,self.videoname[:-4],'keypoints')
        
        if not os.path.exists(savDir):
            os.makedirs(savDir)
        if not os.path.exists(imgSavDir):
            os.makedirs(imgSavDir)
        if not os.path.exists(jsonSavDir):
            os.makedirs(jsonSavDir)
        
        # Load model
        pose_model_loader = PoseModelLoader()
        det_model_loader = YoloLoader()
            
        cap = cv2.VideoCapture(self.videoPath)
        fI_jump=0
        fIList=[]
        personNumList=[]
        resultList=[]
        for fI in tqdm(range(0,self.fcount,self.fps*by_sec)):
            #if fI<fI_jump:
                #continue
            cap.set(cv2.CAP_PROP_POS_FRAMES,fI)
            ret, frame = cap.read()
            framePose = FrameProcessor(frame, det_model_loader, pose_model_loader).start()
            result = framePose.resultNew
            personNum = framePose.det_human_num
            personNumList.append(personNum)
            resultList.append(result)
            fIList.append(fI)
            #fI_jump = fI + self.fps*filter_jump_sec
        
        personNumThre=int(0.9*max(personNumList))
        for fI, personNum, result in tqdm(zip(fIList, personNumList, resultList)):
            if personNum<personNumThre:
                continue
            cap.set(cv2.CAP_PROP_POS_FRAMES,fI)
            ret, frame = cap.read()
            imgSavName=self.videoname[:-4]+'_'+str(fI)+'_'+str(personNum)+'.jpg'
            jsonSavName=self.videoname[:-4]+'_'+str(fI)+'_'+str(personNum)+'_keypoints.json'
            sav_img(os.path.join(imgSavDir, imgSavName), frame)
            with open(os.path.join(jsonSavDir, jsonSavName),'w',encoding='utf-8') as jsonfile:
                json.dump(result, jsonfile)
        cap.release() 

videoPathList = glob.glob(r'/home/ubuntu/Data/Video/class/*.mp4')
for videoPath in videoPathList:
    testVideo = myVideoLoader(videoPath)
    testVideo.auto_extract_frame('/home/ubuntu/Data/autoFrameFilter/',by_sec=1)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/zzx/AlphaPose/SPPE/src/utils/img.py:11: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", 

Loading pose model from ./models/sppe/duc_se.pth


  0%|          | 0/2384 [00:00<?, ?it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2351: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
100%|██████████| 2384/2384 [16:40<00:00,  6.01it/s]
2384it [00:07, 321.24it/s]


In [ ]:
import glob
videoPathList = glob.glob.

In [1]:
import os
from tqdm import tqdm
import time
import numpy as np

import cv2
from myutil import *

from dataloader import FrameProcessor, YoloLoader, PoseModelLoader
def draw_one_alphapose(frame, oneResult, format='coco'):
    if format == 'coco':
        l_pair = [(0, 1), (0, 2), (1, 3), (2, 4),  # Head
            (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),(17, 11), (17, 12),  # Body
            (11, 13), (12, 14), (13, 15), (14, 16)]
        p_color = [(0, 255, 255),(0, 191, 255),(0, 255, 102),(0, 77, 255), (0, 255, 0), 
                   #Nose, LEye, REye, LEar, REar
                  (77,255,255),(77, 255, 204), (77,204,255), (191, 255, 77), (77,191,255), (191, 255, 77), 
                   #LShoulder, RShoulder, LElbow, RElbow, LWrist, RWrist
                  (204,77,255),(77,255,204), (191,77,255), (77,255,191), (127,77,255), (77,255,127), (0, 255, 255)] 
                   #LHip, RHip, LKnee, Rknee, LAnkle, RAnkle, Neck
        line_color = [(0, 215, 255), (0, 255, 204), (0, 134, 255), (0, 255, 50), 
                    (77,255,222), (77,196,255), (77,135,255), (191,255,77), (77,255,77), 
                    (77,222,255), (255,156,127), 
                    (0,127,255), (255,127,77), (0,77,255), (255,77,36)]
    img = frame
    part_line = {}
    kp_preds = np.array(oneResult['keypoints'])
    kp_scores = np.array(oneResult['kp_score'])
    # Draw keypoints
    for n in range(len(kp_scores)):
        if kp_scores[n] <= 0.05:
            continue
        cor_x, cor_y = int(kp_preds[n, 0]), int(kp_preds[n, 1])
        part_line[n] = (cor_x, cor_y)
        cv2.circle(img, (cor_x, cor_y), 4, p_color[n], -1)
    # Draw limbs
    for i, (start_p, end_p) in enumerate(l_pair):
        if start_p in part_line and end_p in part_line:
            start_xy = part_line[start_p]
            end_xy = part_line[end_p]
            cv2.line(img, start_xy, end_xy, line_color[i], 2*(kp_scores[start_p] + kp_scores[end_p]) + 1)
    return img



imgPath = '/home/ubuntu/Data/IMG/4.JPG'
frame = cv2.imread(imgPath)

# Load pose model
pose_model_loader = PoseModelLoader()
det_model_loader = YoloLoader()
startTime = time.time()
framePose = FrameProcessor(frame, det_model_loader, pose_model_loader).start()
#framePose.out_img
#print(time.time()-startTime)
#print(framePose.person_num, framePose.out_img.shape)
#print(framePose.result)
#sav_img( '/home/ubuntu/Data/output/test.JPG',out_img)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading pose model from ./models/sppe/duc_se.pth


/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2351: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [22]:
def plt_img(img, mode=None):
    if mode=='BGR':
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show() 
plt_img(framePose.out_img)


In [2]:
out_img = draw_one_alphapose(frame, framePose.resultNew[0])
sav_img( '/home/ubuntu/Data/output/test_3st.JPG',out_img)

In [3]:
import torch
for human in framePose.resultNew:
    print(np.array(human['kp_score']))
    break

[[0.79628235]
 [0.81959236]
 [0.80907679]
 [0.30262834]
 [0.60456681]
 [0.69666219]
 [0.63703501]
 [0.76484483]
 [0.77513301]
 [0.79137766]
 [0.68912852]
 [0.3997393 ]
 [0.43122238]
 [0.20353107]
 [0.10328924]
 [0.19517055]
 [0.15587234]
 [0.6668486 ]]


In [1]:
a=[1,2,3,4,5,6]
max(a)

6

In [3]:
from tqdm import tqdm
aList=[1,2,3,4,5,6]
bList=[1,2,3,4,5,6]
for a, b in tqdm(zip(aList,bList)):
    pass

6it [00:00, 22389.52it/s]


In [4]:
int(0.95*max(aList))

5

In [6]:
os.path.join('/home','keypoints',self.videoname[:-4],jsonSavname)

NameError: name 'self' is not defined